### Initializing Spark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("noobDE-project-1").enableHiveSupport().getOrCreate()

### Reading data from hdfs

start your hdfs daemons

In [2]:
initial_data = spark.read.csv('hdfs://0.0.0.0:19000/data/covid.csv', header=True, inferSchema=True)

In [3]:
initial_data.show(10)

+----------+-----------+---------+---------+------+
|      Date|    Country|Confirmed|Recovered|Deaths|
+----------+-----------+---------+---------+------+
|2020-01-22|Afghanistan|        0|        0|     0|
|2020-01-23|Afghanistan|        0|        0|     0|
|2020-01-24|Afghanistan|        0|        0|     0|
|2020-01-25|Afghanistan|        0|        0|     0|
|2020-01-26|Afghanistan|        0|        0|     0|
|2020-01-27|Afghanistan|        0|        0|     0|
|2020-01-28|Afghanistan|        0|        0|     0|
|2020-01-29|Afghanistan|        0|        0|     0|
|2020-01-30|Afghanistan|        0|        0|     0|
|2020-01-31|Afghanistan|        0|        0|     0|
+----------+-----------+---------+---------+------+
only showing top 10 rows



In [3]:
aggregated_data = initial_data.groupBy(initial_data['Country']).agg({"Confirmed":"sum", "Recovered":"sum", "Deaths":"sum"}).orderBy(initial_data["Country"])
aggregated_data.show()

+-------------------+--------------+-----------+--------------+
|            Country|sum(Recovered)|sum(Deaths)|sum(Confirmed)|
+-------------------+--------------+-----------+--------------+
|        Afghanistan|      17298848|     919586|      23067096|
|            Albania|      21090178|     510948|      27057392|
|            Algeria|      24301174|    1038079|      35377829|
|            Andorra|       2915536|      39085|       3147489|
|             Angola|       5452593|     164843|       6866562|
|Antigua and Barbuda|        177019|       6369|        213435|
|          Argentina|     657011328|   17152676|     744716612|
|            Armenia|      48998628|    1018755|      54910091|
|          Australia|       8532221|     318531|      11141026|
|            Austria|     124034732|    2266093|     133714775|
|         Azerbaijan|      64772122|     978819|      70854344|
|            Bahamas|       2334485|      59370|       2832808|
|            Bahrain|      45411669|    

In [4]:
renamedAggData = aggregated_data.withColumnRenamed('sum(Recovered)', "Total_Recovered").withColumnRenamed('sum(Confirmed)', "Total_Confirmed").withColumnRenamed('sum(Deaths)', "Total_Deaths")

In [6]:
renamedAggData.show()

+-------------------+---------------+------------+---------------+
|            Country|Total_Recovered|Total_Deaths|Total_Confirmed|
+-------------------+---------------+------------+---------------+
|        Afghanistan|       17298848|      919586|       23067096|
|            Albania|       21090178|      510948|       27057392|
|            Algeria|       24301174|     1038079|       35377829|
|            Andorra|        2915536|       39085|        3147489|
|             Angola|        5452593|      164843|        6866562|
|Antigua and Barbuda|         177019|        6369|         213435|
|          Argentina|      657011328|    17152676|      744716612|
|            Armenia|       48998628|     1018755|       54910091|
|          Australia|        8532221|      318531|       11141026|
|            Austria|      124034732|     2266093|      133714775|
|         Azerbaijan|       64772122|      978819|       70854344|
|            Bahamas|        2334485|       59370|        2832

In [13]:
renamedAggData.createOrReplaceTempView("CovidData")

In [18]:
renamedAggData.repartition(2).write.mode("overwrite").save("./data/covid_data.parquet")

In [7]:
renamedAggData.write.format("jdbc").options(url="jdbc:oracle:thin:@localhost:1521:orcl",driver='oracle.jdbc.driver.OracleDriver',dbtable='CovidData',user='hr',password='hr').mode('append').save()